In [2]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import re
import shutil
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from keras import layers
from keras import models
from keras import regularizers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.metrics import mean_squared_error
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import PIL.Image as Image
from keras.applications import ResNet50V2 
import lime
from keras.applications.imagenet_utils import decode_predictions
from lime import lime_image
from skimage.segmentation import mark_boundaries
import warnings
warnings.simplefilter("ignore")

%matplotlib inline

In [4]:
df = pd.read_csv("data/Crash_data.csv")

In [5]:
df

,OID,PrmaryCollsionType,CaseNumber,UnitsInvolved,CrossStreet,StreetType,DirectionFromIntersection,FeetFromIntersection,CollisionDate,ReportedDate,...,Lon,DUI,AlcoholDrugSuspected,BicyclesInvolved,PedestriansInvolved,RoadDescription,Crash_Time,ESRI_OID,x,y
0,4931,COLLISION W/ MV - FRONT TO REAR,1904154,2,S. BOULDER RD/MANHATTAN CIR.,CITY ST/CNTY RD,W,NaN,4/18/2019 6:00:00 AM,4/18/2019 6:00:00 AM,...,-105.227477,NO,NO,NO,NO,INTERSECTION RELATED,1228,106,-105.227477,39.985895
1,10847,COLLISION W/ MV - FRONT TO REAR,2206289,2,26TH ST/CANYON BLVD,CITY ST/CNTY RD,N,100,7/3/2022 6:00:00 AM,7/3/2022 6:00:00 AM,...,-105.260463,NO,NO,NO,NO,NON-INTERSECTION,1830,107,-105.260463,40.017486
2,11727,COLLISION W/ MV - FRONT TO REAR,2212629,2,ARAPAHOE AVE,OTHER RDWY,,NaN,12/25/2022 7:00:00 AM,12/25/2022 7:00:00 AM,...,-105.263361,NO,NO,NO,NO,NaN,1614,108,-105.263361,40.014816
3,12855,COLLISION W/ MV - FRONT TO SIDE,2307980,2,MOORHEAD AVENUE,NaN,,NaN,8/15/2023 6:00:00 AM,8/17/2023 6:00:00 AM,...,-105.259824,NO,NO,NO,NO,PARKING LOT,2045,109,-105.259824,39.998445
4,12888,COLLISION W/ MV - FRONT TO SIDE,2308220,2,28TH STREET/MAPLETON AVE,STATE HWY,,NaN,8/23/2023 6:00:00 AM,8/23/2023 6:00:00 AM,...,-105.258383,NO,NO,NO,NO,AT INTERSECTION,1330,110,-105.258383,40.025511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13292,8597,COLLISION W/ OBJECT,1,3,,,,,4/12/2021 6:00:00 AM,4/12/2021 6:00:00 AM,...,NaN,NO,NO,NO,NO,EXPRESS/MANAGED/HOV LANE,0900,13447,NaN,NaN
13293,10080,COLLISION W/ OTHER VEH - PARKED MV,2200754,2,CASCADE AVE/20TH ST,CITY ST/CNTY RD,W,96,1/25/2022 7:00:00 AM,1/25/2022 7:00:00 AM,...,-105.267770,NO,NO,NO,NO,ALLEY RELATED,0744,13448,-105.267770,40.001851
13294,12011,COLLISION W/ MV - FRONT TO REAR,2301807,2,CANYON BLVD/28TH ST,STATE HWY,,NaN,2/23/2023 7:00:00 AM,2/23/2023 7:00:00 AM,...,-105.258456,NO,NO,NO,NO,AT INTERSECTION,2050,13449,-105.258456,40.017347
13295,11984,COLLISION W/ MV - FRONT TO SIDE,2301556,2,TABLE MESA DRIVE/S LOOP DRIVE,CITY ST/CNTY RD,,NaN,2/15/2023 7:00:00 AM,2/17/2023 7:00:00 AM,...,-105.234551,NO,NO,NO,NO,AT INTERSECTION,2225,13450,-105.235069,39.986390


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13297 entries, 0 to 13296
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OID                        13297 non-null  int64  
 1   PrmaryCollsionType         13297 non-null  object 
 2   CaseNumber                 13297 non-null  int64  
 3   UnitsInvolved              13297 non-null  int64  
 4   CrossStreet                13290 non-null  object 
 5   StreetType                 11526 non-null  object 
 6   DirectionFromIntersection  13297 non-null  object 
 7   FeetFromIntersection       2628 non-null   object 
 8   CollisionDate              13297 non-null  object 
 9   ReportedDate               13296 non-null  object 
 10  ReportedYear               13296 non-null  float64
 11  ReportedMonth              13296 non-null  float64
 12  ReportedMonthName          13296 non-null  object 
 13  ReportedWeek               13296 non-null  flo

In [8]:
df.describe()

,OID,CaseNumber,UnitsInvolved,ReportedYear,ReportedMonth,ReportedWeek,ReportedDayofWeekNum,ReportedDayofMonth,ReportedHour,NumKilled,NumSeriouslyInjured,NumInjuredOrKilled,Lat,Lon,ESRI_OID,x,y
count,13297.000000,1.329700e+04,13297.000000,13296.000000,13296.00000,13296.000000,13296.000000,13296.000000,13296.000000,13297.000000,13297.000000,13297.000000,12026.000000,12026.000000,13297.000000,12026.000000,12026.000000
mean,6775.936527,1.995658e+06,2.000451,2019.881242,6.87500,13.096495,4.174789,15.601233,13.096495,0.001579,0.027224,0.116643,40.015781,-105.257787,6782.565391,-105.257788,40.015782
std,3867.639926,1.947099e+05,0.592649,1.928615,3.39564,5.050458,1.811704,8.787234,5.050458,0.043333,0.168197,0.357582,0.017732,0.018141,3845.066097,0.018140,0.017731
min,1.000000,1.000000e+00,1.000000,2017.000000,1.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,39.964537,-105.297895,106.000000,-105.297895,39.964537
25%,3439.000000,1.812492e+06,2.000000,2018.000000,4.00000,10.000000,3.000000,8.000000,10.000000,0.000000,0.000000,0.000000,40.005172,-105.272693,3457.000000,-105.272693,40.005172
50%,6792.000000,2.000175e+06,2.000000,2020.000000,7.00000,13.000000,4.000000,16.000000,13.000000,0.000000,0.000000,0.000000,40.014866,-105.258401,6781.000000,-105.258401,40.014866
75%,10124.000000,2.200915e+06,2.000000,2022.000000,10.00000,16.000000,6.000000,23.000000,16.000000,0.000000,0.000000,0.000000,40.023419,-105.248252,10105.000000,-105.248252,40.023419
max,13451.000000,2.373985e+06,44.000000,2023.000000,12.00000,23.000000,7.000000,31.000000,23.000000,2.000000,3.000000,4.000000,40.094401,-105.178496,13451.000000,-105.178496,40.094401
